In [1]:
import numpy as np
import pandas as pd

In [2]:
def convert_units(cmc):
    """Convert from units of log(mol per litre) to log(micro mol per litre)."""
    return np.log10(np.power(10, cmc) * 1e+6)

In [3]:
df = pd.read_html("cosmoplex.html", header=0)[0]
df

,Surfactant,Abbreviation,log10CMC (COSMOplex),log10CMC (experimental)
0,Triethylene glycol monohexyl ether,C6E3,-2.00,-1.00
1,Dimethyl nonylamine oxide,C9C2NO,-1.92,-1.27
2,Octyl Î²-d-glucoside,C8BG1,-2.92,-1.60
3,Triethylene glycol monooctyl ether,C8E3,-3.03,-2.12
4,Octyl Î±-glyceryl ether,C8BGLYE,-2.88,-2.24
5,Octyl glycol ether,C8E1,-3.14,-2.31
6,Triethylene glycol monodecyl ether,C10E3,-3.67,-3.22
7,Dodecyl Î²-d-glucoside,C12BG1,-4.21,-3.72
8,Octoxynol-2,C8(C6H4)E2,-4.15,-3.88
9,Hexaethylene glycol monododecyl ether,C12E6,-5.40,-4.06


In [4]:
df["pred"] = convert_units(df["log10CMC (COSMOplex)"])
df["exp"] = convert_units(df["log10CMC (experimental)"])
df["resid"] = df["exp"] - df["pred"]

In [6]:
rmse = np.sqrt(np.mean(np.square(df["resid"])))
rmse

0.8603836353627374